In [1]:
import json
from bs4 import BeautifulSoup as bs
from tqdm.notebook import tqdm

post_data = json.load(open('ten_thousand_questions/questions_with_bodies.json'))


In [3]:
post_data.keys()

dict_keys(['items', 'has_more', 'quota_max', 'quota_remaining'])

In [4]:
accepted_answers = []
from tqdm.notebook import tqdm

post_data['items'][0]['is_answered']

True

In [5]:
for item in tqdm(post_data['items']):
    if 'accepted_answer_id' in item:
        accepted_answers.append(item)


  0%|          | 0/300 [00:00<?, ?it/s]

In [6]:
len(accepted_answers)

300

In [8]:
from stackapi import StackAPI

SITE = StackAPI('stackoverflow')
aids = [i['accepted_answer_id'] for i in accepted_answers]


In [11]:
answers = SITE.fetch('answers/{ids}', ids=aids[:100], filter='withbody')


{'backoff': 0, 'has_more': False, 'page': 1, 'quota_max': 300, 'quota_remaining': 295, 'total': 0, 'items': [{'owner': {'reputation': 12941, 'user_id': 14289892, 'user_type': 'registered', 'profile_image': 'https://i.stack.imgur.com/mM8Ee.jpg?s=128&g=1', 'display_name': 'Anurag Dabas', 'link': 'https://stackoverflow.com/users/14289892/anurag-dabas'}, 'is_accepted': True, 'score': 3, 'last_activity_date': 1625407195, 'creation_date': 1625407195, 'answer_id': 68245456, 'question_id': 68245443, 'content_license': 'CC BY-SA 4.0', 'body': "<p>try:</p>\n<pre><code>df['counter']=df.index//3\n</code></pre>\n<p>OR</p>\n<p>If you have a custom index then you can use:</p>\n<pre><code>df['counter']=[x//3 for x in range(len(df))]\n</code></pre>\n<p>output of <code>df</code>:</p>\n<pre><code>    key     team    counter\n0   K0      a       0\n1   K1      d       0\n2   K2      w       0\n3   K3      a       1\n4   K4      c       1\n5   K5      s       1\n6   K6      x       2\n7   K7      d       2

In [12]:
final_answers = answers['items']

In [13]:
final_answers.extend(SITE.fetch('answers/{ids}', ids=aids[100:200], filter='withbody')['items'])
print(len(final_answers))

200


In [14]:
final_answers.extend(SITE.fetch('answers/{ids}', ids=aids[200:300], filter='withbody')['items'])
print(len(final_answers))

300


In [16]:
id_to_answer = {
    answer['answer_id']: answer for answer in final_answers
}

In [17]:
for idx in range(len(accepted_answers)):
    accepted_answers[idx]['answer_body'] = id_to_answer[accepted_answers[idx]['accepted_answer_id']]['body']

In [18]:
qids = [i['question_id'] for i in accepted_answers]

In [22]:
question_items = SITE.fetch('questions/{ids}', ids=qids[:100], filter='withbody')['items']
question_items.extend(SITE.fetch('questions/{ids}', ids=qids[100:200], filter='withbody')['items'])
question_items.extend(SITE.fetch('questions/{ids}', ids=qids[200:300], filter='withbody')['items'])
print(len(question_items))

300


In [24]:
id_to_body = {
    q['question_id']: q['body'] for q in question_items
}

In [25]:
for idx in range(len(accepted_answers)):
    accepted_answers[idx]['question_body'] = id_to_body[accepted_answers[idx]['question_id']]

### For Mining Pandas APIs

In [27]:
import requests

urls = json.load(open("pandas_apis.json"))
data_frame_url = urls["df"]

def get_api_list(_url):
    page = requests.get(_url)
    soup = bs(page.content, 'html.parser')
    codes = soup.find_all("code", {"class": "xref py py-obj docutils literal notranslate"})
    apis = []
    for code in codes:
        api_name = code.find("span").text
        if "." in api_name:
            name = api_name[api_name.index(".") + 1:]
            apis.append(name)
            pass
        else:
            apis.append(api_name)
        pass
    return apis

data_frame_apis = get_api_list(data_frame_url)
# print(data_frame_apis)

list_of_apis = []
complete_api_sets = []

for key in urls:
    api_from_url = get_api_list(urls[key])
    list_of_apis.append({
        "url_key": key,
        "url": urls[key],
        "apis": api_from_url
    })
    complete_api_sets.extend(api_from_url)
    pass

# print("Dataframe APIs only")
# print(data_frame_apis)
# print("=" * 100)
# print("Pandas APIS")
complete_api_sets = list(set(complete_api_sets))
# print(complete_api_sets)
# print("=" * 100)
    

In [29]:
# print(data_frame_apis)

def dataframe_api_exists(line):
    for api in data_frame_apis:
        if api in line:
            return True
        pass
    return  False

def api_exists(line):
    for api in complete_api_sets:
        if api in line:
            return True
        pass
    return False


lines = ["hello()", "concat(dsaf)", "index()", "dasfasd"]
for l in lines:
    print(l, dataframe_api_exists(l))

hello() False
concat(dsaf) True
index() True
dasfasd False


In [57]:
import re 
import keyword

keywords = keyword.kwlist

def keyword_exists(line):
    for kw in keywords:
        if (kw + " ") in line or (" " + kw) in line:
            return True
    return False


def min_word_filter(texts, min_word=5):
    filtered_texts = []
    for t in texts:
        if len(t.split()) >= min_word:
            filtered_texts.append(t)
            pass
        pass
    return filtered_texts

def extract_code(text, filters):
    soup = bs(text)
    all_code = [code.text for code in soup.find_all('code')]
    for f in filters:
        all_code = f(all_code)
    return all_code
    pass

def data_frame_exists(code):
    lines = [l.strip() for l in code.split("\n")]
    data_frame_re = "[0-9]+[, \t]+[.]*"
    matches = []
    for l in lines:
        if len(re.findall(data_frame_re, l)) > 0 \
            and not keyword_exists(l) \
            and not dataframe_api_exists(l) \
            and not api_exists(l):
            matches.append(True)
        else:
            matches.append(False)
    return any(matches)

def code_exists(code):
    lines = [l.strip() for l in code.split("\n")]
    matches = []
    for l in lines:
        if dataframe_api_exists(l) or keyword_exists(l) or api_exists(l):
            return True
    return False

def print_formatted_code(code):
    lines = code.split('\n')
    lines = ['\t\t' + l for l in  lines]
    print('\n'.join(lines))
    
def extract_description(html):
    soup = bs(html)
    [code.extract() for code in soup.find_all('code')]
    text = re.sub("[ \n\t]+", " ", soup.text)
    return text
    pass
    
taken_answers = []

for a in accepted_answers:
    title = a['title']
    code_from_body = extract_code(a['question_body'], filters=[min_word_filter])
    code_from_answer = extract_code(a['answer_body'], filters=[min_word_filter])
    data_frames = []
    for cid, c in enumerate(code_from_body):
        is_df = data_frame_exists(c)
        is_code = code_exists(c)
        if is_df and not is_code:
            data_frames.append(c)
        
    taken_code = []
    for cid, c in enumerate(code_from_answer):
        is_df = data_frame_exists(c)
        is_pandas_code = api_exists(c)
        if is_pandas_code and not is_df:
            taken_code.append(c)
    if len(data_frames) == 2 and len(taken_code) > 0:
        a["formatted_input"] = {
            "qid": a['question_id'],
            'link': a['link'],
            "question": {
                "title": a["title"],
                "ques_desc" : extract_description(a['question_body'])
            },
            "io": data_frames,
            "answer" : {
                "ans_desc" : extract_description(a['answer_body']),
                "code": taken_code
            }
        }
        taken_answers.append(a)

print(len(taken_answers))
    
    
    

12


In [60]:
import json 

for a in taken_answers:
    fmt_input = a["formatted_input"]
    print("\"qid\": %s" % fmt_input["qid"])
    print("\"link\": %s" % fmt_input["link"])
    print("\"question\": {")
    print("\t\"title\": %s" % fmt_input["question"]["title"])
    print("\t\"desc\": %s" % fmt_input["question"]["ques_desc"])
    print("}")
    print("\"io\": {")
    print("\t\"Frame-1\": ")
    print_formatted_code(fmt_input["io"][0])
    print("\t\"Frame-2\":")
    print_formatted_code(fmt_input["io"][1])
    print("}")
    print("\"answer\": {")
    print("\t\"desc\": %s", fmt_input["answer"]["ans_desc"])
    print("\t\"code-snippets\": [")
    for t in fmt_input["answer"]["code"]:
        print_formatted_code(t)
        print("\t\t" + ("-" * 70))
    print("\t]")
    print("}")
    print("=" * 100)
    print("\n")

"qid": 68243146
"link": https://stackoverflow.com/questions/68243146/replace-zero-with-value-of-an-other-column-using-pandas
"question": {
	"title": replace zero with value of an other column using pandas
	"desc": I have a dataframe df1: I want to replace 0 in the id column with value from ref column of the same row So it will become: 
}
"io": {
	"Frame-1": 
		    ref   Name   id  Score
		  8400   John    0     12
		  3840  Peter  414      0
		  7400  David  612     64
		  5200  Karen    0      0
		
	"Frame-2":
		   ref    Name   id   Score
		  8400   John  8400     12
		  3840  Peter  414      0
		  7400  David  612     64
		  5200  Karen 5200      0
		
}
"answer": {
	"desc": %s via : OR via numpy's : 
	"code-snippets": [
		#import numpy as np
		df['id']=np.where(df['id'].eq(0),df['ref'],df['id'])
		
		----------------------------------------------------------------------
	]
}


"qid": 68231389
"link": https://stackoverflow.com/questions/68231389/compare-two-columns-that-contains-time

In [59]:
all_answers_file = open("outputs/all_accepted_answers_with_all_details.json", 'w')
json.dump(obj=accepted_answers, fp=all_answers_file, indent=4)
all_answers_file.close()

taken_answer_file = open("outputs/taken_answers_with_all_details.json", "w")
json.dump(obj=taken_answers, fp=taken_answer_file, indent=4)
taken_answer_file.close()

In [68]:
# questions = []
# for pi in range(1, 21):
#     d = SITE.fetch(
#             "search/advanced", 
#             q="pandas,dataframe", 
#             accepted=True, 
#             tagged="pandas;dataframe;python", 
#             page=pi, 
#             filter="withbody"
#         )['items']
#     for itm in d:
#         if 'accepted_answer_id' in itm.keys() and 'pandas' in itm['tags'] and 'dataframe' in itm['tags']:
#             questions.append(itm)
#     print(len(questions))

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000


In [69]:
# import os
# os.makedirs('ten_thousand_questions', exist_ok=True)
# ten_thousand_questions = open('ten_thousand_questions/questions_with_bodies.json', 'w')
# json.dump(questions, fp=ten_thousand_questions, indent=4)
# ten_thousand_questions.close()